In [ ]:
#INIT
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account
from utils.utils_chain import nominated_amount
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_current_tokens_for_address, get_token_details_for_address, get_all_token_nonces_details_for_account, dec_to_padded_hex
from utils.utils_tx import ESDTToken, NetworkProviders
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.dex_proxy_contract import DexProxyContract, DexProxyClaimRewardsEvent, DexProxyEnterFarmEvent, DexProxyExitFarmEvent, DexProxyRemoveLiquidityEvent, DexProxyAddLiquidityEvent

context = Context()

In [ ]:
#ADDRESS SETUP
user = Account(pem_file="~/Documents/mx-sdk-py-exchange/wallets/C1.pem")
user.address = Address("")
user2 = Account(pem_file="")
user2.address = ""
sc_owner = Account(pem_file="~/Documents/mx-sdk-py-exchange/wallets/C10.pem")
sc_owner.address = Address("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
user.sync_nonce(context.network_provider.proxy)
# opt_original_caller.sync_nonce(context.network_provider.proxy)
sc_owner.sync_nonce(context.network_provider.proxy)

In [ ]:
#CONTRACTS INIT
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

In [ ]:
#CLAIM REWARDS
txhash = fees_collector_contract.claim_rewards(user, context.network_provider.proxy)
used_api = context.network_provider.api

CLAIM ORIGINAL CALLER

In [ ]:
txhash = fees_collector_contract.claim_rewards_original_caller(user, context.network_provider.proxy, user)

CLAIM BOOSTED REWARDS

In [ ]:
txhash = fees_collector_contract.claim_boosted_rewards(user, context.network_provider.proxy)
used_api = context.network_provider.api

tokens_list = {}
tokens_list = get_current_tokens_for_address(user.address, context.network_provider.proxy)
if tokens_list.keys == 0:
    print("No tokens")
    raise
print(tokens_list.items())

# Get the transaction details
tx = used_api.get_transaction(txhash)
sender = tx.sender.bech32()

ops = tx.raw_response['data']
print(ops)


CHECK ENERGY AFTER CLAIM

In [ ]:
current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)

get_total_energy_for_week_hex = fees_collector_contract.get_total_energy_for_week(context.network_provider.proxy , current_week)
print(get_total_energy_for_week_hex)
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.to_bech32(), context.network_provider.proxy ,current_week)
print(get_user_energy_hex)
user_energy_before, staking_stats_before = fees_collector_contract.get_all_stats(context.network_provider.proxy, user.address.to_bech32(), current_week)


CHECK CLAIM

In [ ]:
op_to_look_for = {
        "action": "MultiESDTNFTTransfer",
        "sender": user.address,
        "receiver": user.address,
        "value": 0
    }
api = context.network_provider.api
op = fees_collector_contract.get_tx_op(txhash, op_to_look_for, api)
print(op)

CHECK SHADOWFORK SCENARIO

In [ ]:
import time
from typing import cast
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
from utils.utils_generic import TestStepConditions


SECONDSDELAY = 5

user.sync_nonce(context.network_provider.proxy)
proxy1 = ProxyNetworkProvider("https://proxy-shadowfork-one.elrond.ro")
api1 = ApiNetworkProvider("https://express-api-shadowfork-one.elrond.ro")
sf1 = NetworkProviders(api1, proxy1)

txhash = fees_collector_contract.claim_boosted_rewards(user, context.network_provider.proxy)
print(txhash)
user.sync_nonce(proxy2)
txhash2 = fees_collector_contract.claim_rewards(user, proxy2)
print(txhash2)

time.sleep(SECONDSDELAY)

current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
get_total_rewards_hex1 = fees_collector_contract.get_total_rewards_for_week(context.network_provider.proxy , current_week)
get_total_rewards_hex2 = fees_collector_contract.get_total_rewards_for_week(context.network_provider.proxy , current_week)

token1 = get_total_rewards_hex1[0]
token2 = get_total_rewards_hex2[0]
print(token1)

expected_token1 = ESDTToken(token1[0][1], int((token1[1][1])), token1[2][1]) 
expected_token2 = ESDTToken(token2[0][1], int((token2[1][1])), token2[2][1]) 

operations = context.network_provider.get_tx_operations(txhash)
operations2 = sf1.get_tx_operations(txhash2)
print(operations)
print(operations2)

teststep = TestStepConditions()
teststep.add_condition(context.network_provider.check_simple_tx_status(txhash),""),
teststep.add_condition(context.network_provider.check_for_transfer_operation(txhash, expected_token1,
                                                                                sender=fees_collector_contract.address,
                                                                                destination=user.address),
                           "claimed tokens")
context.network_provider.check_complex_tx_status
teststep.assert_conditions()

teststep2 = TestStepConditions()
teststep2.add_condition(sf1.check_simple_tx_status(txhash2),""),
teststep2.add_condition(sf1.check_for_transfer_operation(txhash2, expected_token2,
                                                                                 destination=user.address.bech32()),
                           "claimed tokens")
sf1.check_complex_tx_status
teststep2.assert_conditions()



allowExternalClaim

In [ ]:
txhash = fees_collector_contract.allow_external_claim(user, context.network_provider.proxy, True)
tx = used_api.get_transaction(txhash)
data = tx.raw_response['data']
print(data)

External Claim

In [ ]:
# opt_original_caller.address
txhash = fees_collector_contract.claim_for_user(user, user2.address, context.network_provider.proxy )

In [ ]:
#UPGRADE CONTRACT
from contracts.fees_collector_contract import FeesCollectorContract

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

# get_account_keys_online(router_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/pre_upgrade_router.json")
fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy,
                                 config.FEES_COLLECTOR_BYTECODE_PATH_V3, [], True)

============ VIEWS =================

In [ ]:
#GET CURRENT WEEK
current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
print(current_week)

In [ ]:
#GET USER ENERGY FOR WEEK
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.bech32(), context.network_provider.proxy , current_week)
print(get_user_energy_hex)

In [ ]:
#GET TOTAL ENERGY FOR WEEK
get_total_energy_for_week_hex = fees_collector_contract.get_total_energy_for_week(context.network_provider.proxy , current_week)
print(get_total_energy_for_week_hex)

In [ ]:
#GET TOTAL REWARDS FOR WEEK
current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)

get_total_rewards_hex = fees_collector_contract.get_total_rewards_for_week(context.network_provider.proxy , current_week-2)
print(get_total_rewards_hex)

GET ALL STATS

In [ ]:
current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
get_user_energy_hex = fees_collector_contract.get_user_energy_for_week(user.address.bech32(), context.network_provider.proxy , current_week)

all_stats = fees_collector_contract.get_all_stats(context.network_provider.proxy, user.address.to_bech32, current_week)
print(all_stats)